# Create a Marquee Backcasted Portfolio with GS Quant

The Marquee Portfolio Service provides a powerful framework for uploading portfolio positions and retrieving analytics including historical performance, factor risk exposure, ESG analytics, and more. GS Quant makes operating the suite of Portfolio Service API's intuitive and fast.

## Step 1: Authenticate and Initialize Your Session

First you will import the necessary modules and add your client id and client secret.

In [ ]:
import datetime as dt
import warnings

from gs_quant.datetime import business_day_offset
from gs_quant.markets.portfolio import Portfolio
from gs_quant.markets.portfolio_manager import PortfolioManager
from gs_quant.markets.position_set import PositionSet
from gs_quant.markets.report import FactorRiskReport, ThematicReport
from gs_quant.models.risk_model import FactorRiskModel
from gs_quant.session import GsSession, Environment
from gs_quant.entities.entitlements import Entitlements, EntitlementBlock, User


client = None
secret = None

## External users must fill in their client ID and secret below and comment out the line below

# client = 'ENTER CLIENT ID'
# secret = 'ENTER CLIENT SECRET'

GsSession.use(Environment.PROD, client_id=client, client_secret=secret)
warnings.filterwarnings("ignore", category=RuntimeWarning)

print('GS Session initialized.')

## Step 2: Create the Portfolio

The first step is to create a new, empty portfolio in Marquee.

In [ ]:
portfolio = Portfolio(name='My New Backcasted Portfolio')
portfolio.save()

print(f"Created portfolio '{portfolio.name}' with ID: {portfolio.id}")

Once your portfolio has been saved to Marquee, the `PortfolioManager` class allows users to interact with their Marquee portfolios directly from GS Quant. We will be using `PortfolioManager` to update portfolio positions, entitlements, update custom AUM, and run reports.

In [ ]:
pm = PortfolioManager(portfolio.id)

## Step 3: Define Portfolio Entitlements (Optional)

By default, an application will have all entitlement permissions to a portfolio it makes.
However, if you would like to share the portfolio with others, either Marquee users or other
applications, you will need to specify them in the entitlements parameter of the portfolio.
Let's walk through how we convert a list of admin and viewer emails into an `Entitlements` object:

*Note: If you would like to see this portfolio on your Marquee webpage, you'll need to add your account
email address into the `portfolio_admin_emails` list*

In [ ]:
portfolio_admin_emails = ['LIST OF ADMIN EMAILS']
portfolio_viewer_emails = ['LIST OF VIEWER EMAILS']

admin_entitlements = EntitlementBlock(users=User.get_many(emails=portfolio_admin_emails))
view_entitlements = EntitlementBlock(users=User.get_many(emails=portfolio_viewer_emails))

entitlements = Entitlements(view=view_entitlements, admin=admin_entitlements)

print(f'Entitlements:\n{entitlements.to_dict()}')

pm.set_entitlements(entitlements)

print(f"Updated entitlements for '{portfolio.name}'")

## Step 4: Define Portfolio Positions

Portfolio positions in Marquee are stored on a holding basis, when means you only upload positions for days where you are rebalancing your portfolio. Depending on your quantity type, take the following example positions:

### Uploading Positions Using Shares
Portfolio quantity type uses share counts.

In [ ]:
positions = [
    {'identifier': 'AAPL UW', 'quantity': 25},
    {'identifier': 'GS UN', 'quantity': 50},
    {'identifier': 'META UW', 'quantity': 25},
    {'identifier': 'AMZN UN', 'quantity': 50},
    {'identifier': 'MSFT UW', 'quantity': 25},
    {'identifier': 'AZN UW', 'quantity': 50},
]

### Uploading Positions Using Exposures
Portfolio quantity type uses notional values.

In [ ]:
positions = [
    {'identifier': 'AAPL UW', 'notional': 25000},
    {'identifier': 'GS UN', 'notional': 50000},
    {'identifier': 'META UW', 'notional': 25000},
    {'identifier': 'AMZN UN', 'notional': 50000},
    {'identifier': 'MSFT UW', 'notional': 25000},
    {'identifier': 'AZN UW', 'notional': 50000},
]

For uploading using quantities or exposures, resolve the positions in your portfolio as of last business day.

In [ ]:
portfolio_position_sets = [
    PositionSet.from_dicts(date=business_day_offset(dt.date.today(), -1, roll='forward'), positions=positions)
]

for pos_set in portfolio_position_sets:
    pos_set.resolve()

print("All positions have been resolved and saved.")

### Uploading Positions Using Weights
Portfolio quantity type uses weight values.

In [ ]:
positions = [
    {'identifier': 'AAPL UW', 'weight': 0.25},
    {'identifier': 'GS UN', 'weight': 0.25},
    {'identifier': 'META UW', 'weight': 0.25},
    {'identifier': 'AMZN UN', 'weight': 0.25},
]

For uploading using weights, resolve the positions in your portfolio as of last business day and using a desired target notional.

In [ ]:
portfolio_position_sets = [
    PositionSet.from_dicts(
        date=business_day_offset(dt.date.today(), -1, roll='forward'), reference_notional=15000, positions=positions
    )
]

for pos_set in portfolio_position_sets:
    pos_set.resolve()

print("All positions have been resolved and saved.")

## Price Positions (Weights & Exposures)
When uploading positions by weights or exposures they must priced and converted to share quantities before being uploaded to the portfolio.
Select the currency you want to price your positions with. If you do not specify, default is USD.

In [ ]:
currency = 'USD'

for pos_set in portfolio_position_sets:
    pos_set.price(currency)

## Define Currency for portfolio

In [ ]:
from gs_quant.common import Currency

pm = PortfolioManager(portfolio.id)
pm.set_currency(Currency.USD)  # Set the currency using PortfolioManager

## Step 5: Upload the positions onto the portfolio:

In [ ]:
pm.update_positions(portfolio_position_sets)

print(f"Successfully updated positions for portfolio {pm.portfolio_id}\n")
print(f"Latest updated set of positions for {pm.portfolio_id}: \n{pm.get_latest_position_set().get_positions()}")

## Step 6: Create Factor Risk and/or Thematic Reports (Optional)

By default, creating a portfolio will automatically create a corresponding performance report for it as well.
If you would like to create a factor risk and/or thematic report (more documentation on reports found [here](https://developer.gs.com/p/docs/services/portfolio/programmatic-access/reports/))
for it as well, run the following:

In [ ]:
risk_model_id = 'RISK MODEL ID'

# Add a factor risk report
risk_report = FactorRiskReport(risk_model_id=risk_model_id)
risk_report.set_position_source(portfolio.id)
risk_report.save()

# Add a thematic report
thematic_report = ThematicReport()
thematic_report.set_position_source(portfolio.id)
thematic_report.save()

print('All portfolio reports created.')

### Quick Tip!

*Explore the different factor risk models available in Marquee in our [Data Catalog](https://marquee.gs.com/s/discover/data-services/catalog?Category=Factor+Risk+Model)*

*Premium clients get access to many more risk models (including premium vendors like MSCI Barra),
while non-premium clients get access to a limited suite of models. To see which models you have access to,
simply run the following:*

In [ ]:
risk_models = FactorRiskModel.get_many(limit=100)
for risk_model in risk_models:
    print(f'{risk_model.name}: {risk_model.id}\n')

## Step 7: Schedule Reports

Now, it's schedule all the portfolio reports. Once this is done and reports are completed, you can programmatically retrieve factor risk and attribution data for your portfolio.

When scheduling reports, you have two options:
- Backcast the report: Take the earliest date with positions in the portfolio / basket and run the report on the positions held then with a start date before the earliest position date and an end date
 of the earliest position date. This option is ideal for snapshot portfolios.
- Do not backcast the report: Set the start date as a date that has positions in the portfolio or basket and an end date after that (best practice is to set it to T-1). In this case the
 report will run on positions held as of each day in the date range. This option is ideal for historical portfolios.

In [ ]:
pm.schedule_reports(backcast=True)

print('All portfolio reports scheduled.')

Note: Cannot Schedule backcasted report with no position sets on the report job end date.